In [2]:
import numpy as np
import pandas as pd
# Load the datasets
df = pd.read_csv("https://raw.githubusercontent.com/wongwara/Jobseeker_Baymax/main/dataset/listings2019_2022_salary_04052023.csv")

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3902 entries, 0 to 3901
Data columns (total 52 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   jobId                  3902 non-null   int64  
 1   jobTitle               3902 non-null   object 
 2   jobClassification      3902 non-null   object 
 3   jobSubClassification   3902 non-null   object 
 4   advertiserName         3902 non-null   object 
 5   advertiserId           3902 non-null   int64  
 6   companyId              1067 non-null   float64
 7   companyName            1476 non-null   object 
 8   companyRating          1067 non-null   float64
 9   listingDate            3902 non-null   object 
 10  expiryDate             3902 non-null   object 
 11  teaser                 3374 non-null   object 
 12  nation                 3902 non-null   object 
 13  state                  3902 non-null   object 
 14  city                   3902 non-null   object 
 15  area

In [5]:
df = df[['jobClassification','state','teaser','nation','workType','salary_string','isRightToWorkRequired','desktopAdTemplate',
         'Python','SQL','R','Tableau','SAS','Matlab','Hadoop','Spark','Java', 'Scala','recruiter']]

In [6]:
len(df['jobClassification'].unique())

25

In [7]:
df['state'].unique()

array(['New South Wales', 'Australian Capital Territory', 'Victoria',
       'Western Australia', 'Queensland', 'Northern Territories',
       'South Australia', 'Tasmania', 'UK & Ireland', 'Overseas',
       'Northern Territory'], dtype=object)

In [8]:
df = df[df['nation'].str.contains('Australia')]

In [9]:
df['state'] = df['state'].replace({'Northern Territories': 'Northern Territory'})

In [10]:
df['state'].unique()

array(['New South Wales', 'Australian Capital Territory', 'Victoria',
       'Western Australia', 'Queensland', 'Northern Territory',
       'South Australia', 'Tasmania'], dtype=object)

In [11]:
# Import OrdinalEncoder from sklearn.preprocessing
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,OneHotEncoder

state_cats = [['Australian Capital Territory', 'South Australia',
       'Western Australia']]
Label = LabelEncoder()
# Use our trained encoder to transform this column
df['state_encoded'] = Label.fit_transform(df[['state']])

# display the columns
df[['state','state_encoded']]

df['state']= df['state_encoded']

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3897 entries, 0 to 3901
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   jobClassification      3897 non-null   object
 1   state                  3897 non-null   int64 
 2   teaser                 3369 non-null   object
 3   nation                 3897 non-null   object
 4   workType               3897 non-null   object
 5   salary_string          1512 non-null   object
 6   isRightToWorkRequired  3411 non-null   object
 7   desktopAdTemplate      3203 non-null   object
 8   Python                 3897 non-null   int64 
 9   SQL                    3897 non-null   int64 
 10  R                      3897 non-null   int64 
 11  Tableau                3897 non-null   int64 
 12  SAS                    3897 non-null   int64 
 13  Matlab                 3897 non-null   int64 
 14  Hadoop                 3897 non-null   int64 
 15  Spark                

In [13]:
df = df.drop(['nation'],axis =1)

In [14]:
df.isna().sum() 

jobClassification           0
state                       0
teaser                    528
workType                    0
salary_string            2385
isRightToWorkRequired     486
desktopAdTemplate         694
Python                      0
SQL                         0
R                           0
Tableau                     0
SAS                         0
Matlab                      0
Hadoop                      0
Spark                       0
Java                        0
Scala                       0
recruiter                   0
state_encoded               0
dtype: int64

In [15]:
df["salary_string"].describe

<bound method NDFrame.describe of 0                                               NaN
1                                             Super
2                        $90000 - $120000 per annum
3                             $90000 - $110000 p.a.
4                                               NaN
                           ...                     
3897    Open to Quote (Sydney or Canberra Location)
3898                                            NaN
3899                                Desirable Rates
3900                                            NaN
3901                                            NaN
Name: salary_string, Length: 3897, dtype: object>

In [16]:
df.head()

,jobClassification,state,teaser,workType,salary_string,isRightToWorkRequired,desktopAdTemplate,Python,SQL,R,Tableau,SAS,Matlab,Hadoop,Spark,Java,Scala,recruiter,state_encoded
0,Science & Technology,1,My client is a leading Australian-owned and ra...,Full Time,NaN,f,\n \n \n Senior Method Development Im...,0,0,1,0,0,0,0,0,0,0,1,1
1,Information & Communication Technology,0,The Data Scientist will be responsible for mon...,Full Time,Super,t,\n \n \n \n Pricing Data Scientist...,0,0,1,0,1,0,0,0,0,0,1,0
2,Information & Communication Technology,1,Fantastic organisation seeks experienced Insig...,Full Time,$90000 - $120000 per annum,f,\n \n \n \n Insights Analyst – Onl...,0,1,1,0,0,0,0,0,0,0,1,1
3,Banking & Financial Services,1,This role requires an individual with strong c...,Full Time,$90000 - $110000 p.a.,f,\n \n \n \n Credit Risk Analyst \n...,0,1,1,0,1,0,0,0,0,0,1,1
4,Information & Communication Technology,1,"Our client, a large Sydney based telecommunica...",Contract/Temp,NaN,f,\n \n \n \n Data Scientist \n ...,1,1,0,1,0,0,0,0,0,0,0,1


In [17]:
import re

def extract_salary(text):    
    range_pattern = r'([\d\.]+) *- *\$?([\d\.]+)'
    range_matches = re.search(range_pattern, text.replace(",", ""))
    if range_matches:
        lo, hi = range_matches.groups()
        salary_range = (float(lo), float(hi))
    else:
        salary_range = None
    
    return salary_range


In [18]:
df = df[df['salary_string'].notna()]

In [19]:
df["salary_string"] = df["salary_string"].apply(extract_salary)

In [20]:
df.head(5)

,jobClassification,state,teaser,workType,salary_string,isRightToWorkRequired,desktopAdTemplate,Python,SQL,R,Tableau,SAS,Matlab,Hadoop,Spark,Java,Scala,recruiter,state_encoded
1,Information & Communication Technology,0,The Data Scientist will be responsible for mon...,Full Time,None,t,\n \n \n \n Pricing Data Scientist...,0,0,1,0,1,0,0,0,0,0,1,0
2,Information & Communication Technology,1,Fantastic organisation seeks experienced Insig...,Full Time,"(90000.0, 120000.0)",f,\n \n \n \n Insights Analyst – Onl...,0,1,1,0,0,0,0,0,0,0,1,1
3,Banking & Financial Services,1,This role requires an individual with strong c...,Full Time,"(90000.0, 110000.0)",f,\n \n \n \n Credit Risk Analyst \n...,0,1,1,0,1,0,0,0,0,0,1,1
7,Banking & Financial Services,1,One of Australia's leading financial service p...,Full Time,"(110000.0, 120000.0)",t,\n \n \n Data Analytics Recruitment Sol...,1,1,1,0,1,0,0,0,0,0,1,1
8,Information & Communication Technology,6,Awesome opportunity for a highly skilled and e...,Full Time,None,t,\n \n \n Senior Data Scientist \n \n...,1,0,0,0,0,0,1,0,0,0,1,6


In [21]:
df = df[df['salary_string'] != (None,None)]

In [22]:
df.head()

,jobClassification,state,teaser,workType,salary_string,isRightToWorkRequired,desktopAdTemplate,Python,SQL,R,Tableau,SAS,Matlab,Hadoop,Spark,Java,Scala,recruiter,state_encoded
1,Information & Communication Technology,0,The Data Scientist will be responsible for mon...,Full Time,None,t,\n \n \n \n Pricing Data Scientist...,0,0,1,0,1,0,0,0,0,0,1,0
2,Information & Communication Technology,1,Fantastic organisation seeks experienced Insig...,Full Time,"(90000.0, 120000.0)",f,\n \n \n \n Insights Analyst – Onl...,0,1,1,0,0,0,0,0,0,0,1,1
3,Banking & Financial Services,1,This role requires an individual with strong c...,Full Time,"(90000.0, 110000.0)",f,\n \n \n \n Credit Risk Analyst \n...,0,1,1,0,1,0,0,0,0,0,1,1
7,Banking & Financial Services,1,One of Australia's leading financial service p...,Full Time,"(110000.0, 120000.0)",t,\n \n \n Data Analytics Recruitment Sol...,1,1,1,0,1,0,0,0,0,0,1,1
8,Information & Communication Technology,6,Awesome opportunity for a highly skilled and e...,Full Time,None,t,\n \n \n Senior Data Scientist \n \n...,1,0,0,0,0,0,1,0,0,0,1,6


In [23]:
df = df[df['salary_string'].notna()]

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 816 entries, 2 to 3893
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   jobClassification      816 non-null    object
 1   state                  816 non-null    int64 
 2   teaser                 695 non-null    object
 3   workType               816 non-null    object
 4   salary_string          816 non-null    object
 5   isRightToWorkRequired  704 non-null    object
 6   desktopAdTemplate      694 non-null    object
 7   Python                 816 non-null    int64 
 8   SQL                    816 non-null    int64 
 9   R                      816 non-null    int64 
 10  Tableau                816 non-null    int64 
 11  SAS                    816 non-null    int64 
 12  Matlab                 816 non-null    int64 
 13  Hadoop                 816 non-null    int64 
 14  Spark                  816 non-null    int64 
 15  Java                  

In [25]:
# df.to_csv('df_salary_cleaned.csv')

In [26]:
df['isRightToWorkRequired'].unique()

array(['f', 't', nan], dtype=object)

In [27]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode

# Define a function to get mode of the column
def get_mode(column):
    return mode(column, nan_policy='omit')[0][0]

# Replace the NaN values with the mode of the column
mode_val = get_mode(df['isRightToWorkRequired'])
df['isRightToWorkRequired'] = df['isRightToWorkRequired'].fillna(mode_val)

# Create the LabelEncoder object
le = LabelEncoder()

# Fit the LabelEncoder object to the column
le.fit(df['isRightToWorkRequired'])

# Transform the column to label encoding
column = le.transform(df['isRightToWorkRequired'])

fill missing value in isRightToworkRequired with mode

In [28]:
# Import OrdinalEncoder from sklearn.preprocessing
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,OneHotEncoder

The state, workType columns use labelEncoder and jobClassification use Ordinalencoder

In [29]:
df['isRightToWorkRequired'].unique()

array(['f', 't'], dtype=object)

In [30]:
df['isRightToWorkRequired'] = df['isRightToWorkRequired'].replace({'f': 0, 't': 1})

In [31]:
df['workType'].unique()

array(['Full Time', 'Contract/Temp', 'Part Time', 'Casual/Vacation',
       'Full time', 'Part time'], dtype=object)

In [32]:
df['workType'] = df['workType'].replace({'Full time': 'Full Time', 'Part time': 'Part Time'})

In [33]:
df['workType'].unique()

array(['Full Time', 'Contract/Temp', 'Part Time', 'Casual/Vacation'],
      dtype=object)

In [34]:
workType_cats = [['Full Time', 'Contract/Temp', 'Part Time', 'Casual/Vacation']]
Label = LabelEncoder()
# Use our trained encoder to transform this column
df['workType_encoded'] = Label.fit_transform(df[['workType']])

# display the columns
df[['workType','workType_encoded']]

,workType,workType_encoded
2,Full Time,2
3,Full Time,2
7,Full Time,2
10,Full Time,2
11,Full Time,2
...,...,...
3854,Full Time,2
3864,Full Time,2
3880,Full Time,2
3891,Contract/Temp,1


In [35]:
df['workType']= df['workType_encoded']

In [36]:
jobClassification_cats = [['Information & Communication Technology',
       'Banking & Financial Services', 'Science & Technology',
       'Education & Training', 'Government & Defence',
       'Consulting & Strategy', 'Healthcare & Medical',
       'Human Resources & Recruitment', 'Marketing & Communications',
       'Retail & Consumer Products', 'Administration & Office Support',
       'Accounting', 'Insurance & Superannuation',
       'Mining, Resources & Energy', 'Real Estate & Property',
       'Manufacturing, Transport & Logistics', 'Engineering']]
jobClassification_cats_enc =OrdinalEncoder(categories=jobClassification_cats)
# Use our trained encoder to transform this column
df['jobClassification_encoded'] = jobClassification_cats_enc.fit_transform(df[['jobClassification']])

# display the columns
df[['jobClassification','jobClassification_encoded']]

df['jobClassification'] = df['jobClassification_encoded']

In [37]:
select_fulltime = df.loc[df['workType'] == 2]
select_fulltime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 639 entries, 2 to 3893
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   jobClassification          639 non-null    float64
 1   state                      639 non-null    int64  
 2   teaser                     541 non-null    object 
 3   workType                   639 non-null    int64  
 4   salary_string              639 non-null    object 
 5   isRightToWorkRequired      639 non-null    int64  
 6   desktopAdTemplate          538 non-null    object 
 7   Python                     639 non-null    int64  
 8   SQL                        639 non-null    int64  
 9   R                          639 non-null    int64  
 10  Tableau                    639 non-null    int64  
 11  SAS                        639 non-null    int64  
 12  Matlab                     639 non-null    int64  
 13  Hadoop                     639 non-null    int64 

In [38]:
# df.to_csv('df_salary_fulltime.csv')